In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

from tqdm import tqdm
from shutil import copyfile
import zipfile
import os
import shutil

In [ ]:
with zipfile.ZipFile('../input/dogs-vs-cats/train.zip','r') as zip_ref:
    zip_ref.extractall('./')
    
with zipfile.ZipFile('../input/dogs-vs-cats/test1.zip','r') as zip_ref:
    zip_ref.extractall('./')

In [ ]:
os.makedirs('./PetImages')
os.makedirs('./PetImages/Cat')
os.makedirs('./PetImages/Dog')

In [ ]:
src_path = './train/'

# seed(1)
# val_ratio = 0.25

for file in tqdm(os.listdir(src_path)):
    src = src_path + file
    
    dest_dir = './PetImages/'
        
    if file.startswith('cat'):
        dest = dest_dir + 'Cat/' + file
        copyfile(src,dest)
    else:
        dest = dest_dir + 'Dog/' + file
        copyfile(src,dest)
    
shutil.rmtree('train')

In [ ]:
image_size = (180,180)
batch_size = 32

train_ds = tf.keras.preprocessing.image_dataset_from_directory(
    "PetImages",
    validation_split = 0.2,
    subset = 'training',
    seed = 1337,
    image_size = image_size,
    batch_size = batch_size
)

val_ds = tf.keras.preprocessing.image_dataset_from_directory(
    "PetImages",
    validation_split = 0.2,
    subset = 'validation',
    seed = 1337,
    image_size = image_size,
    batch_size = batch_size
)

In [ ]:
plt.figure(figsize =(10,15))
for images,labels in train_ds.take(1):
    for i in range(9):
        ax = plt.subplot(3,3,i+1)
        plt.imshow(images[i].numpy().astype('uint8'))
        plt.title(int(labels[i]))
        plt.axis('off')

In [ ]:
data_augmentation = keras.Sequential([
    layers.experimental.preprocessing.RandomFlip('horizontal'),
    layers.experimental.preprocessing.RandomRotation(0.1),
])

In [ ]:
plt.figure(figsize = (10,10))
for images, _ in train_ds.take(1):
    for i in range(9):
        augmented_images = data_augmentation(images)
        ax = plt.subplot(3,3,i+1)
        plt.imshow(augmented_images[0].numpy().astype('uint8'))
        plt.axis('off')

In [ ]:
train_ds = train_ds.prefetch(buffer_size=32)
val_ds = val_ds.prefetch(buffer_size = 32)

In [ ]:
def make_model(input_shape,num_classes):
    inputs = keras.Input(shape = input_shape)
    #Image Augmentation Block
    x = data_augmentation(inputs)
    
    #Entry block
    x = layers.experimental.preprocessing.Rescaling(1.0/255)(x)
    x = layers.Conv2D(32,3,strides=2,padding='same')(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation('relu')(x)
    
    x = layers.Conv2D(64,3,padding = 'same')(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation('relu')(x)
    
    previous_block_activation = x #Set Aside Residual
    
    for size in [128,256,512,728]:
        x = layers.Activation('relu')(x)
        x = layers.SeparableConv2D(size, 3, padding = 'same')(x)
        x = layers.BatchNormalization()(x)
        
        x = layers.Activation('relu')(x)
        x = layers.SeparableConv2D(size, 3, padding = 'same')(x)
        x = layers.BatchNormalization()(x)
        
        x = layers.MaxPooling2D(3, strides = 2, padding = 'same')(x)
        
        #Project Residual
        residual = layers.Conv2D(size, 1, strides = 2, padding = 'same')(previous_block_activation)
        x = layers.add([x,residual])
        previous_block_activation = x #Setting Aside next residual
    
    x = layers.SeparableConv2D(1024,3,padding='same')(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation('relu')(x)
    
    x = layers.GlobalAveragePooling2D()(x)
    if num_classes == 2:
        activation = 'sigmoid'
        units = 1
    else:
        activation = 'softmax'
        units = num_classes
        
    x = layers.Dropout(0.5)(x)
    outputs = layers.Dense(units, activation = activation)(x)
    
    return keras.Model(inputs,outputs)
    
model = make_model(input_shape=image_size + (3,), num_classes=2)
keras.utils.plot_model(model, show_shapes=True)

In [ ]:
epochs = 25

model.compile(
    optimizer = keras.optimizers.Adam(1e-3),
    loss = 'binary_crossentropy',
    metrics = ['accuracy']
)

model.fit( 
    train_ds,
    epochs = epochs,
    validation_data= val_ds
)

In [ ]:
model.evaluate(val_ds)

In [ ]:
img = keras.preprocessing.image.load_img(
    "PetImages/Cat/cat.0.jpg", target_size=image_size
)
img_array = keras.preprocessing.image.img_to_array(img)
img_array = tf.expand_dims(img_array, 0)  # Create batch axis
plt.imshow(plt.imread('PetImages/Cat/cat.0.jpg'))
plt.axis('off')
predictions = model.predict(img_array)
score = predictions[0]
print(
    "This image is %.2f percent cat and %.2f percent dog."
    % (100 * (1 - score), 100 * score)
)